In [1]:
source("http://bioconductor.org/biocLite.R")
biocLite("rhdf5")

Bioconductor version 3.3 (BiocInstaller 1.22.3), ?biocLite for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.3 (BiocInstaller 1.22.3), R 3.3.1 (2016-06-21).
Installing package(s) ‘rhdf5’
Old packages: 'BiocParallel', 'curl', 'DBI', 'DESeq2', 'knitr', 'limma', 'R6',
  'S4Vectors', 'stringr'


In [2]:
devtools::install_github("pachterlab/sleuth")

from URL https://api.github.com/repos/pachterlab/sleuth/zipball/master
Installing sleuth
Installing data.table
Installing chron
'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL '/tmp/RtmpwKIUHz/devtools23da42049686/chron'  \
  --library='/home/ilya/R/x86_64-pc-linux-gnu-library/3.3' --install-tests 

'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL '/tmp/RtmpwKIUHz/devtools23da51f5a0bf/data.table'  \
  --library='/home/ilya/R/x86_64-pc-linux-gnu-library/3.3' --install-tests 

Installing dplyr
Installing assertthat
'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL '/tmp/RtmpwKIUHz/devtools23da4cb8ff2d/assertthat'  \
  --library='/home/ilya/R/x86_64-pc-linux-gnu-library/3.3' --install-tests 

Installing BH
'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL '/tmp/RtmpwKIUHz/devtools23da61e5f1ae/BH'  \
  --libra

In [3]:
library("sleuth")

Loading required package: ggplot2
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [18]:
base_dir <- "../results/kallisto"
sample_id <- dir(file.path(base_dir), pattern='^lo')
sample_id

[1] "lo01" "lo02" "lo03" "lo04" "lo05" "lo06" "lo07" "lo08" "lo09" "lo10"
[11] "lo11" "lo12"

In [33]:
kal_dirs <- sapply(sample_id, function(id) file.path(base_dir, id))
kal_dirs

lo01                       lo02 
"../results/kallisto/lo01" "../results/kallisto/lo02" 
                      lo03                       lo04 
"../results/kallisto/lo03" "../results/kallisto/lo04" 
                      lo05                       lo06 
"../results/kallisto/lo05" "../results/kallisto/lo06" 
                      lo07                       lo08 
"../results/kallisto/lo07" "../results/kallisto/lo08" 
                      lo09                       lo10 
"../results/kallisto/lo09" "../results/kallisto/lo10" 
                      lo11                       lo12 
"../results/kallisto/lo11" "../results/kallisto/lo12"

In [27]:
dir(kal_dirs)

[1] "abundance.h5"  "abundance.h5"  "abundance.h5"  "abundance.h5" 
 [5] "abundance.h5"  "abundance.h5"  "abundance.h5"  "abundance.h5" 
 [9] "abundance.h5"  "abundance.h5"  "abundance.h5"  "abundance.h5" 
[13] "abundance.tsv" "abundance.tsv" "abundance.tsv" "abundance.tsv"
[17] "abundance.tsv" "abundance.tsv" "abundance.tsv" "abundance.tsv"
[21] "abundance.tsv" "abundance.tsv" "abundance.tsv" "abundance.tsv"
[25] "run_info.json" "run_info.json" "run_info.json" "run_info.json"
[29] "run_info.json" "run_info.json" "run_info.json" "run_info.json"
[33] "run_info.json" "run_info.json" "run_info.json" "run_info.json"

In [20]:
gsy_samples <- c(sample_id[seq(1, length(sample_id), 4)], sample_id[seq(3, length(sample_id), 4)])
gsy_samples

[1] "lo01" "lo05" "lo09" "lo03" "lo07" "lo11"

In [25]:
gsy_dirs <- c(kal_dirs[seq(1, length(kal_dirs), 4)], kal_dirs[seq(3, length(kal_dirs), 4)])
gsy_dirs

lo01                       lo05 
"../results/kallisto/lo01" "../results/kallisto/lo05" 
                      lo09                       lo03 
"../results/kallisto/lo09" "../results/kallisto/lo03" 
                      lo07                       lo11 
"../results/kallisto/lo07" "../results/kallisto/lo11"

In [34]:
s2c <- data.frame(sample=gsy_samples, condition=c('wt', 'wt', 'wt', 'gsy', 'gsy', 'gsy'))
s2c <- dplyr::mutate(s2c, path = gsy_dirs)
s2c

,sample,condition,path
1,lo01,wt,../results/kallisto/lo01
2,lo05,wt,../results/kallisto/lo05
3,lo09,wt,../results/kallisto/lo09
4,lo03,gsy,../results/kallisto/lo03
5,lo07,gsy,../results/kallisto/lo07
6,lo11,gsy,../results/kallisto/lo11


In [35]:
so <- sleuth_prep(s2c, ~ condition)

reading in kallisto results
......
normalizing est_counts
19296 targets passed the filter
normalizing tpm
merging in metadata
normalizing bootstrap samples
summarizing bootstraps


In [36]:
so <- sleuth_fit(so)

fitting measurement error models
shrinkage estimation
Adding missing grouping variables: `x_group`
computing variance of betas


In [37]:
so <- sleuth_fit(so, ~1, 'reduced')

fitting measurement error models
shrinkage estimation
Adding missing grouping variables: `x_group`
computing variance of betas


In [38]:
#so <- sleuth_test(so, which_beta='conditionwt')
so <- sleuth_lrt(so, 'reduced', 'full')

In [39]:
models(so)

[  full  ]
formula:  ~condition 
coefficients:
	(Intercept)
 	conditionwt
[  reduced  ]
formula:  ~1 
coefficients:
	(Intercept)


In [41]:
source("http://bioconductor.org/biocLite.R")
biocLite("biomaRt")

Bioconductor version 3.3 (BiocInstaller 1.22.3), ?biocLite for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.3 (BiocInstaller 1.22.3), R 3.3.1 (2016-06-21).
Installing package(s) ‘biomaRt’
also installing the dependencies ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘RSQLite’, ‘S4Vectors’, ‘XML’, ‘AnnotationDbi’

Old packages: 'boot', 'MASS', 'spatial'


In [42]:
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
  dataset = "celegans_gene_ensembl",
  host = 'ensembl.org')

In [43]:
t2g <- biomaRt::getBM(attributes = c("ensembl_transcript_id", "ensembl_gene_id",
    "external_gene_name"), mart = mart)
t2g <- dplyr::rename(t2g, target_id = ensembl_transcript_id,
  ens_gene = ensembl_gene_id, ext_gene = external_gene_name)
so <- sleuth_prep(s2c, ~ condition, target_mapping = t2g)

reading in kallisto results
......
normalizing est_counts
19296 targets passed the filter
normalizing tpm
merging in metadata
normalizing bootstrap samples
summarizing bootstraps


In [44]:
so <- sleuth_fit(so)

fitting measurement error models
shrinkage estimation
Adding missing grouping variables: `x_group`
computing variance of betas


In [46]:
so <- sleuth_fit(so, ~1, 'reduced')

fitting measurement error models
shrinkage estimation
Adding missing grouping variables: `x_group`
computing variance of betas


In [47]:
so <- sleuth_lrt(so, 'reduced', 'full')

In [51]:
so <- sleuth_wt(so, which_beta = 'conditionwt')

In [52]:
sleuth_live_settings(test_type='wt')

$test_type
[1] "wt"

In [ ]:
sleuth_live(so)


Listening on http://127.0.0.1:42427
Warning message:
: Removed 8 rows containing missing values (geom_point).Warning message:
: Removed 8 rows containing missing values (geom_point).

In [10]:
write.table(res,file='../kallisto/results.tsv', sep='\t')

In [ ]:
sleuth_live(so)

Loading required package: shiny

Listening on http://127.0.0.1:7861


In [ ]:
res <- sleuth_results(so, 'conditionwt')